## 이미지를 무작위로 뽑아서 확인해 보겠습니다.


In [31]:
from collections import namedtuple
import json
from os import path
import os
import random
from typing import (
    Any, List, Set, Tuple, Callable, Iterable, Mapping, NewType
)

from PIL import Image, ImageShow

`json` 애노테이션을 읽어서 이미지 경로와 일치 여부를 먼저 확인하겠습니다.


그전에 일단 경로 확인


In [4]:
print(os.getcwd())
print(os.listdir('dataset'))

/home/elice/project-cyberdyne/ai
['validation-labels-src', 'training-labels-src', 'training', 'NanumGothic.ttf', 'validation']


라벨을 읽어서 매치되는 이미지가 없으면 표시합니다.


`*_images` 형식은 아래와 같이 할 계획입니다.

```py
box = tuple[tuple[int, int], tuple[int, int]]
List[tuple[pathname, List[box]]]
```


가끔 폴리곤이란 녀석들이 있어서 변환 함수가 필요합니다.


In [10]:
Box = NewType('Box', Tuple[Tuple[int, int], Tuple[int, int]])

def polygon_to_box(verts: Iterable[Mapping[str, str]]) -> Box:
    '''``POLYGON`` 타입 드로잉을 ``BOX`` 형식으로 변환해 반환합니다.'''
    as_nums = [
        tuple(map(int, list(vert.values())[0].split(','))) for vert in verts
    ]
    box = (
        (min(x for x, y in as_nums), min(y for x, y in as_nums)),
        (max(x for x, y in as_nums), max(y for x, y in as_nums)),
    )
    return box


In [28]:
ImageInfo = namedtuple('ImageInfo', ['path', 'boxes'])

def get_imageinfo(top: str) -> List[ImageInfo]:
    images = []

    for stem, branches, leaves in os.walk(top):
        for leaf in leaves:
            if path.splitext(leaf)[1].lower() == '.json':
                pathname = path.join(stem, leaf)
                json_in = open(pathname, 'r', encoding='utf-8')
                label = json.load(json_in)

                # 이미지 상대 경로를 구합니다.
                prefix = path.dirname(path.relpath(pathname, top))
                image_path = path.join(prefix, label['FILE NAME'])

                # 박스 리스트를 만듭니다.
                boxes = []
                for box_n in range(int(label['BoundingCount'])):
                    boxinfo = label['Bounding'][box_n]
                    if boxinfo['Drawing'] == 'POLYGON':
                        box = polygon_to_box(boxinfo['PolygonPoint'])
                    else:
                        box = map(
                            int, 
                            (
                                boxinfo['x1'],
                                boxinfo['y1'],
                                boxinfo['x2'],
                                boxinfo['y2'],
                            )
                        )
                    boxes.append(box)

                images.append(ImageInfo(image_path, boxes))

    return images

In [29]:
train_images = get_imageinfo('./dataset/training-labels-src')
val_images = get_imageinfo('./dataset/validation-labels-src')

이미지가 실제로 있는지 확인해 보겠습니다. 하나는 이미 알고 있습니다.


In [33]:
train_images_missing = set()
for imageinfo in train_images:
    pathname = path.join('./dataset/training', imageinfo.path)
    if not path.exists(pathname):
        # print(pathname)
        train_images_missing.add(imageinfo.path)

val_images_missing = set()
for imageinfo in val_images:
    pathname = path.join('./dataset/validation', imageinfo.path)
    if not path.exists(pathname):
        # print(pathname)
        val_images_missing.add(imageinfo.path)


In [34]:
print(len(train_images_missing))
print(len(val_images_missing))


1
0


In [35]:
print(train_images_missing)
print(val_images_missing)

{'캔류/맥주캔/22_X006_C013_1015/22_X006_C013_1015_0.jpg'}
set()


### ~~빠진 파일들~~

~~라벨 -> 이미지 순으로 매치했을 때 빠진 파일이 엄청나게 많다는 것을 알았습니다.~~

~~빠진 파일은 training 에서 27000개, validation 에서 14000 개 가량입니다.~~

- ~~training: 의류/외투, 상의, 하의~~
- ~~validation: 유리병, 가구류, 나무 (하위는 다양)~~

수정: *상대경로 계산을 잘못해서 그랬던 것 같습니다.*

이미지 -> 라벨 순으로 매치하면 어떤 파일이 빠지는지도 봐야 할 것 같습니다.

혹시 이미지 파일 이름이 그냥 단순하게 틀린 것도 있을지 모릅니다.


In [32]:
def get_missing_json(top: str) -> Set[str]:
    result = set()
    for stem, branches, leaves in os.walk(top):
        for leaf in leaves:
            result.add(path.join(stem, leaf))

    return result
